In [67]:
import pandas as pd
import dtale
import numpy as np

In [68]:
normal = pd.read_csv('../../datasets/InSDN_DatasetCSV/Normal_data.csv')
ovs = pd.read_csv('../../datasets/InSDN_DatasetCSV/OVS.csv')
metasploitable = pd.read_csv('../../datasets/InSDN_DatasetCSV/metasploitable-2.csv')

df = pd.concat([normal, ovs, metasploitable], ignore_index=True)

print(df)

                                         Flow ID          Src IP  Src Port  \
0       185.127.17.56-192.168.20.133-443-53648-6   185.127.17.56       443   
1       185.127.17.56-192.168.20.133-443-53650-6  192.168.20.133     53650   
2         192.168.20.133-192.168.20.2-35108-53-6  192.168.20.133     35108   
3         192.168.20.133-192.168.20.2-35108-53-6    192.168.20.2        53   
4       154.59.122.74-192.168.20.133-443-60900-6  192.168.20.133     60900   
...                                          ...             ...       ...   
343884  192.168.3.130-200.175.2.130-41966-4444-6   192.168.3.130     41966   
343885  192.168.3.130-200.175.2.130-41967-4444-6   192.168.3.130     41967   
343886   192.168.3.130-200.175.2.130-139-44791-6   200.175.2.130     44791   
343887  192.168.3.130-200.175.2.130-41966-4444-6   192.168.3.130     41966   
343888  192.168.3.130-200.175.2.130-41967-4444-6   192.168.3.130     41967   

                Dst IP  Dst Port  Protocol        Timestamp  Fl

### Missing datas

useless here because no missing datas

In [69]:
from sklearn.impute import SimpleImputer # to handle missing data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# print("Missing datas : " )
# print(df.isnull().sum())


# print(df[['Flow Duration', 'Flow Byts/s', 'Fwd Pkt Len Mean', 'Flow IAT Mean']].head())
# column = ['Flow Duration', 'Flow Byts/s', 'Fwd Pkt Len Mean', 'Flow IAT Mean']
# df[column] = df[column].fillna(df[column].median())

# column = ["SYN Flag Cnt", "Tot Fwd Pkts", "Fwd Act Data Pkts"]
# df.loc[:, column] = df.loc[:,column].fillna(0)

# print(df[['Flow Duration', 'Flow Byts/s', 'Fwd Pkt Len Mean', 'Flow IAT Mean']].head())
# print("Missing datas : " )
# print(df.isnull().sum())

print("Missing datas by columns :\n", df.isnull().sum())

num_cols_median = ['Flow Duration', 'Flow Byts/s', 'Fwd Pkt Len Mean', 'Flow IAT Mean']
df[num_cols_median] = df[num_cols_median].fillna(df[num_cols_median].median())

cols_fill0 = ["SYN Flag Cnt", "Tot Fwd Pkts", "Fwd Act Data Pkts"]
df[cols_fill0] = df[cols_fill0].fillna(0)
df = df.drop(columns=['Flow ID', 'Src IP', 'Dst IP', 'Timestamp'])

Missing datas by columns :
 Flow ID      0
Src IP       0
Src Port     0
Dst IP       0
Dst Port     0
            ..
Idle Mean    0
Idle Std     0
Idle Max     0
Idle Min     0
Label        0
Length: 84, dtype: int64


### Separing datas and labels

In [70]:
X = df.drop(columns=['Label']) # without labels
Y = df['Label'] # just labels

### Encoding categorical data
Difference between Label and OneHot : \
                                      - Label gives a number in int for each line (simple but the model could misunderstand the difference between numbers (priorities issues))\
                                      - OneHot encodes in binary columns (no hierarchy between column but a lot of columns if they are a lot of categories to label)

In [71]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_protocol = encoder.fit_transform(X[['Protocol']])
protocol_cols = encoder.get_feature_names_out(['Protocol'])
encoded_protocol_df = pd.DataFrame(encoded_protocol, columns=protocol_cols, index=X.index) # convert in dataframe
X = pd.concat([X.drop(columns=['Protocol']), encoded_protocol_df], axis=1) #concatenate with the encoded version of protocol

print(X)

# Y_encoder = LabelEncoder()
# Y = Y_encoder.fit_transform(Y)

Y = Y.apply(lambda x: 0 if x == 'Normal' else 1) # because we have several types of attacks and we wants bianaries Y

print(Y)

        Src Port  Dst Port  Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  \
0            443     53648         245230            44            40   
1          53650       443        1605449           107           149   
2          35108        53          53078             5             5   
3             53     35108           6975             1             1   
4          60900       443         190141            13            16   
...          ...       ...            ...           ...           ...   
343884     41966      4444         273133             2             3   
343885     41967      4444         267969             6             7   
343886     44791       139        1552555             6             5   
343887     41966      4444         272141             2             3   
343888     41967      4444         270361             6             7   

        TotLen Fwd Pkts  TotLen Bwd Pkts  Fwd Pkt Len Max  Fwd Pkt Len Min  \
0                124937             1071     

### Splitting into training set and test set
We are now splitting the dataset
The train set that has the full data to train and the test set which has only 3 columns for testing on smaller samples

In [72]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
print(X_train)

        Src Port  Dst Port  Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  \
325042        80     44370          31423             0             2   
298378     38739        80           3838             4             4   
268971         0         0             24             0             2   
253598         0         0             38             0             2   
253091         0         0             13             0             2   
...          ...       ...            ...           ...           ...   
122579     43468        80          59812             2             6   
304137        80     33495       62379457             3             7   
152315     41830        80          34759             2             6   
117952     37776        80          15896             2             6   
305711        80     41927       63291589             3             7   

        TotLen Fwd Pkts  TotLen Bwd Pkts  Fwd Pkt Len Max  Fwd Pkt Len Min  \
325042                0                0     

### Feature Scaling
To have the same scale between each columns (for instance, `Flow Duration` is way bigger than `Tot Fwd Pkts` and the model could misinterpret it and gives more importance to the Income column)

In [73]:
scaler = StandardScaler()

numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])
print("X_train :\n", X_train, "\n")
print("X_test :\n", X_test)

X_train :
         Src Port  Dst Port  Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  \
325042 -0.924461  2.234924      -0.306988     -0.003916     -0.036937   
298378  0.709191 -0.490033      -0.308252     -0.001762     -0.019123   
268971 -0.927842 -0.494955      -0.308426     -0.003916     -0.036937   
253598 -0.927842 -0.494955      -0.308426     -0.003916     -0.036937   
253091 -0.927842 -0.494955      -0.308427     -0.003916     -0.036937   
...          ...       ...            ...           ...           ...   
122579  0.909030 -0.490033      -0.305688     -0.002839     -0.001309   
304137 -0.924461  1.565836       2.548979     -0.002300      0.007599   
152315  0.839811 -0.490033      -0.306835     -0.002839     -0.001309   
117952  0.668497 -0.490033      -0.307699     -0.002839     -0.001309   
305711 -0.924461  2.084618       2.590761     -0.002300      0.007599   

        TotLen Fwd Pkts  TotLen Bwd Pkts  Fwd Pkt Len Max  Fwd Pkt Len Min  \
325042        -0.009755        -0.

### Prepare data for Deep Learning (convert datas into float32)

In [74]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')


Y_train = np.array(Y_train).astype('float32')
Y_test = np.array(Y_test).astype('float32')

X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)



### CNN training

In [ ]:
CNN_model = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    Conv1D(32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

CNN_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

CNN_model.summary()

history = CNN_model.fit(X_train, Y_train, epochs=20, batch_size=64, validation_data=(X_test, Y_test))

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_7 (Conv1D)               │ (None, 79, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_7 (MaxPooling1D)  │ (None, 39, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 39, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 1248)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 64)             │        79,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,129 (313.00 KB)

 Trainable params: 80,129 (313.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
3762/3762 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.9897 - loss: 0.0351 - val_accuracy: 0.9973 - val_loss: 0.0092
Epoch 2/20
3762/3762 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9973 - loss: 0.0113 - val_accuracy: 0.9979 - val_loss: 0.0072
Epoch 3/20
3762/3762 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.9979 - loss: 0.0085 - val_accuracy: 0.9994 - val_loss: 0.0048
Epoch 4/20
3762/3762 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.9986 - loss: 0.0073 - val_accuracy: 0.9994 - val_loss: 0.0052
Epoch 5/20
3762/3762 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.9990 - loss: 0.0071 - val_accuracy: 0.9995 - val_loss: 0.0034
Epoch 6/20
3762/3762 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.9992 - loss: 0.0047 - val_accuracy: 0.9995 - val_loss: 0.0034
Epoch 7/20
3762/3762 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.9992 - loss: 0.0042 - val_accuracy: 0.9995 - val_loss: 0.0038
Epoch 8/20
3762/3762 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9993 - loss: 0

### LSTM training

In [76]:
LSTM_model = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

LSTM_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

LSTM_model.summary()


history = LSTM_model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_data=(X_test, Y_test))


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,121 (82.50 KB)

 Trainable params: 21,121 (82.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
3762/3762 ━━━━━━━━━━━━━━━━━━━━ 88s 23ms/step - accuracy: 0.9730 - loss: 0.0856 - val_accuracy: 0.9957 - val_loss: 0.0239
Epoch 2/5
3762/3762 ━━━━━━━━━━━━━━━━━━━━ 86s 23ms/step - accuracy: 0.9954 - loss: 0.0262 - val_accuracy: 0.9959 - val_loss: 0.0226
Epoch 3/5
3762/3762 ━━━━━━━━━━━━━━━━━━━━ 88s 23ms/step - accuracy: 0.9963 - loss: 0.0194 - val_accuracy: 0.9962 - val_loss: 0.0131
Epoch 4/5
3762/3762 ━━━━━━━━━━━━━━━━━━━━ 85s 23ms/step - accuracy: 0.9962 - loss: 0.0162 - val_accuracy: 0.9967 - val_loss: 0.0108
Epoch 5/5
3762/3762 ━━━━━━━━━━━━━━━━━━━━ 87s 23ms/step - accuracy: 0.9966 - loss: 0.0122 - val_accuracy: 0.9967 - val_loss: 0.0109


Let's save our models

In [ ]:
CNN_model.save("cnn_insdn_model.keras")
LSTM_model.save("lstm_insdn_model.keras")

In [80]:
prediction = LSTM_model.predict(X_test)

   1/3224 ━━━━━━━━━━━━━━━━━━━━ 1:10 22ms/step

3224/3224 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step
